In [1]:
# multi-head attention TFT

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, Flatten, MultiHeadAttention
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.callbacks import EarlyStopping
from itertools import product
import gc

# Load dataset
file_path = 'data/basic.csv'
data = pd.read_csv(file_path)

# Remove unnecessary columns
data = data.drop(columns=['last_name, first_name'])

# Handle missing values (e.g., replace with 0)
data = data.fillna(0)

# Extract player_ids that exist in 2020, 2021, 2022, 2023
data_2019 = data[data['year'] == 2019]
data_2020 = data[data['year'] == 2020]
data_2021 = data[data['year'] == 2021]
data_2022 = data[data['year'] == 2022]
data_2023 = data[data['year'] == 2023]

player_ids_2019 = set(data_2019['player_id'].unique())
player_ids_2020 = set(data_2020['player_id'].unique())
player_ids_2021 = set(data_2021['player_id'].unique())
player_ids_2022 = set(data_2022['player_id'].unique())
player_ids_2023 = set(data_2023['player_id'].unique())

common_player_ids = player_ids_2019 & player_ids_2020 & player_ids_2021 & player_ids_2022 & player_ids_2023

# Extract data for common player_ids
common_data = data[data['player_id'].isin(common_player_ids)]

# Extract data for the years 2020, 2021, and 2022
final = common_data[common_data['year'].isin([2019, 2020, 2021, 2022])]
final = final.sort_values(by=['player_id', 'year'])

# Select necessary columns (excluding year)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# Separate independent and dependent variables
X = final[features].values
y = final[target].values

# Data scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Convert to time series data format
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 4  # Set sequence length
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# Set training data to be the same as the entire data
X_train, y_train = X_seq, y_seq

# Set hyperparameters
state_sizes = [10, 20, 40, 80, 160, 240, 320]
dropout_rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.9]
minibatch_sizes = [32, 64, 128]
learning_rates = [0.0001, 0.001, 0.01]
max_gradient_norms = [0.01, 1, 100.0]
num_heads = [1, 2, 3, 4]

# Generate hyperparameter combinations
hyperparameter_combinations = list(product(state_sizes, dropout_rates, minibatch_sizes, learning_rates, max_gradient_norms, num_heads))

best_rmse = float('inf')
best_mae = float('inf')
best_mape = float('inf')
best_params = None

# Define TFT model
class TFTModel(Model):
    def __init__(self, seq_length, feature_dim, num_heads, ff_dim, state_size, dropout_rate):
        super(TFTModel, self).__init__()
        self.multi_head_attention1 = MultiHeadAttention(num_heads=num_heads, key_dim=feature_dim)
        self.layer_norm1 = LayerNormalization()
        self.multi_head_attention2 = MultiHeadAttention(num_heads=num_heads, key_dim=feature_dim)
        self.layer_norm2 = LayerNormalization()
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(feature_dim)
        ])
        self.flatten = Flatten()
        self.dense1 = Dense(state_size, activation="relu")
        self.dropout1 = Dropout(dropout_rate)
        self.dense2 = Dense(state_size // 4, activation="relu")
        self.dense3 = Dense(1, activation="linear")

    def call(self, inputs):
        attn_output1 = self.multi_head_attention1(inputs, inputs)
        out1 = self.layer_norm1(inputs + attn_output1)
        attn_output2 = self.multi_head_attention2(out1, out1)
        out2 = self.layer_norm2(out1 + attn_output2)
        ffn_output = self.ffn(out2)
        flat_output = self.flatten(ffn_output)
        dense_output1 = self.dense1(flat_output)
        drop_output1 = self.dropout1(dense_output1)
        dense_output2 = self.dense2(drop_output1)
        return self.dense3(dense_output2)

# Hyperparameter tuning
for state_size, dropout_rate, minibatch_size, learning_rate, max_gradient_norm, num_heads in hyperparameter_combinations:
    tf.keras.backend.clear_session()
    
    # Create model instance
    model = TFTModel(seq_length=seq_length, feature_dim=X_train.shape[2], num_heads=num_heads, ff_dim=32, state_size=state_size, dropout_rate=dropout_rate)
    
    # Compile
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=max_gradient_norm)
    model.compile(
        loss="mse",
        optimizer=optimizer,
        metrics=["accuracy"]
    )

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Train model
    history = model.fit(X_train, y_train, epochs=500, batch_size=minibatch_size, validation_split=0.2, verbose=0, callbacks=[early_stopping])
    
    # Filter 2023 data
    data_19 = common_data[common_data['year'] == 2019]

    # Scale 2023 data
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # Convert to time series data format (use 2022, 2021, 2020 data to predict 2023)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # Predict 2023 data
    y_pred_scaled = model.predict(X_2019_seq)

    # Inverse scale
    y_pred = scaler_y.inverse_transform(y_pred_scaled)

    # Restore actual 2023 p_era values for comparison
    y_test_actual = data_19[target].values

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test_actual[:len(y_pred)], y_pred))

    # Calculate MAE
    mae = mean_absolute_error(y_test_actual[:len(y_pred)], y_pred)
    
    # Calculate MAPE
    mape = mean_absolute_percentage_error(y_test_actual[:len(y_pred)], y_pred)

    if rmse < best_rmse:
        best_rmse = rmse
        best_mae = mae
        best_mape = mape
        best_params = (state_size, dropout_rate, minibatch_size, learning_rate, max_gradient_norm, num_heads)

    print(f"Params: {state_size, dropout_rate, minibatch_size, learning_rate, max_gradient_norm, num_heads}, RMSE: {rmse}, MAE: {mae}, MAPE: {mape}")

    del model, history, y_pred_scaled, y_pred  # Explicitly delete variables
    tf.keras.backend.clear_session()
    gc.collect()
    
print(f"Best RMSE: {best_rmse}, Best MAE: {best_mae}, Best MAPE: {best_mape}, Best Params: {best_params}")

2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 0.01, 1), RMSE: 1.1072030269774464, MAE: 0.8646944769601973, MAPE: 0.2260047045118423
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 0.01, 2), RMSE: 0.7778516948488151, MAE: 0.5321248643360442, MAPE: 0.13311903005064574
2/2 [==============================] - 0s 3ms/step
Params: (10, 0.1, 32, 0.0001, 0.01, 3), RMSE: 1.2542445275538805, MAE: 0.9227464286864749, MAPE: 0.21491670709553118
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 0.01, 4), RMSE: 0.7365875199442931, MAE: 0.548953820940048, MAPE: 0.13298383935028607
2/2 [==============================] - 0s 8ms/step
Params: (10, 0.1, 32, 0.0001, 1, 1), RMSE: 1.2568639515164355, MAE: 0.9515309144580175, MAPE: 0.2256765515625116
2/2 [==============================] - 0s 384us/step
Params: (10, 0.1, 32, 0.0001, 1, 2), RMSE: 1.326516383826914, MAE: 0.9698928872365801, MAPE: 0.21608240972896176
2/2

In [1]:
# basic TFT

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, Flatten
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.callbacks import EarlyStopping
from itertools import product
import gc

# Load dataset
file_path = 'data/basic.csv'
data = pd.read_csv(file_path)

# Remove unnecessary columns
data = data.drop(columns=['last_name, first_name'])

# Handle missing values (e.g., replace with 0)
data = data.fillna(0)

# Extract player_ids that exist in 2019, 2020, 2021, 2022, 2023
data_2019 = data[data['year'] == 2019]
data_2020 = data[data['year'] == 2020]
data_2021 = data[data['year'] == 2021]
data_2022 = data[data['year'] == 2022]
data_2023 = data[data['year'] == 2023]

player_ids_2019 = set(data_2019['player_id'].unique())
player_ids_2020 = set(data_2020['player_id'].unique())
player_ids_2021 = set(data_2021['player_id'].unique())
player_ids_2022 = set(data_2022['player_id'].unique())
player_ids_2023 = set(data_2023['player_id'].unique())

common_player_ids = player_ids_2019 & player_ids_2020 & player_ids_2021 & player_ids_2022 & player_ids_2023

# Extract data for common player_ids
common_data = data[data['player_id'].isin(common_player_ids)]

# Extract data for the years 2019, 2020, 2021, and 2022
final = common_data[common_data['year'].isin([2019, 2020, 2021, 2022])]
final = final.sort_values(by=['player_id', 'year'])

# Select necessary columns (excluding year)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# Separate independent and dependent variables
X = final[features].values
y = final[target].values

# Data scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Convert to time series data format
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 4  # Set sequence length
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# Set training data to be the same as the entire data
X_train, y_train = X_seq, y_seq

# Set hyperparameters
state_sizes = [10, 20, 40, 80, 160, 240, 320]
dropout_rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.9]
minibatch_sizes = [32, 64, 128]
learning_rates = [0.0001, 0.001, 0.01]
max_gradient_norms = [0.01, 1, 100.0]
num_heads = [1, 2, 3, 4]

# Generate hyperparameter combinations
hyperparameter_combinations = list(product(state_sizes, dropout_rates, minibatch_sizes, learning_rates, max_gradient_norms, num_heads))

best_rmse = float('inf')
best_mae = float('inf')
best_mape = float('inf')
best_params = None

# Define model
class BasicTFTModel(Model):
    def __init__(self, seq_length, feature_dim, ff_dim, state_size, dropout_rate):
        super(BasicTFTModel, self).__init__()
        self.layer_norm1 = LayerNormalization()
        self.ffn1 = Dense(ff_dim, activation="relu")
        self.layer_norm2 = LayerNormalization()
        self.ffn2 = Dense(feature_dim)
        self.flatten = Flatten()
        self.dense1 = Dense(state_size, activation="relu")
        self.dropout1 = Dropout(dropout_rate)
        self.dense2 = Dense(state_size // 4, activation="relu")
        self.dense3 = Dense(1, activation="linear")

    def call(self, inputs):
        x = self.layer_norm1(inputs)
        x = self.ffn1(x)
        x = self.layer_norm2(x)
        x = self.ffn2(x)
        flat_output = self.flatten(x)
        dense_output1 = self.dense1(flat_output)
        drop_output1 = self.dropout1(dense_output1)
        dense_output2 = self.dense2(drop_output1)
        return self.dense3(dense_output2)

# Hyperparameter tuning
for state_size, dropout_rate, minibatch_size, learning_rate, max_gradient_norm, num_heads in hyperparameter_combinations:
    tf.keras.backend.clear_session()
    
    # Create model instance
    model = BasicTFTModel(seq_length=seq_length, feature_dim=X_train.shape[2], ff_dim=32, state_size=state_size, dropout_rate=dropout_rate)
    
    # Compile
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=max_gradient_norm)
    model.compile(
        loss="mse",
        optimizer=optimizer,
        metrics=["accuracy"]
    )

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Train model
    history = model.fit(X_train, y_train, epochs=500, batch_size=minibatch_size, validation_split=0.2, verbose=0, callbacks=[early_stopping])
    
    # Filter 2019 data
    data_19 = common_data[common_data['year'] == 2019]

    # Scale 2019 data
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # Convert to time series data format
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # Predict 2019 data
    y_pred_scaled = model.predict(X_2019_seq)

    # Inverse scale
    y_pred = scaler_y.inverse_transform(y_pred_scaled)

    # Restore actual 2019 p_era values for comparison
    y_test_actual = data_19[target].values

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test_actual[:len(y_pred)], y_pred))

    # Calculate MAE
    mae = mean_absolute_error(y_test_actual[:len(y_pred)], y_pred)
    
    # Calculate MAPE
    mape = mean_absolute_percentage_error(y_test_actual[:len(y_pred)], y_pred)

    if rmse < best_rmse:
        best_rmse = rmse
        best_mae = mae
        best_mape = mape
        best_params = (state_size, dropout_rate, minibatch_size, learning_rate, max_gradient_norm, num_heads)

    print(f"Params: {state_size, dropout_rate, minibatch_size, learning_rate, max_gradient_norm, num_heads}, RMSE: {rmse}, MAE: {mae}, MAPE: {mape}")

    del model, history, y_pred_scaled, y_pred  # Explicitly delete variables
    tf.keras.backend.clear_session()
    gc.collect()
    
print(f"Best RMSE: {best_rmse}, Best MAE: {best_mae}, Best MAPE: {best_mape}, Best Params: {best_params}")



2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 0.01, 1), RMSE: 1.0192013440424832, MAE: 0.7084621529352099, MAPE: 0.17629963218795422
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 0.01, 2), RMSE: 2.2770762895801075, MAE: 1.5961345207880413, MAPE: 0.3945009954024432
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 0.01, 3), RMSE: 0.9592476178632046, MAE: 0.6713724370986697, MAPE: 0.16521477142924976
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 0.01, 4), RMSE: 1.4799031877769968, MAE: 1.1232024919419061, MAPE: 0.26485929838294353
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 1, 1), RMSE: 0.8095965373279763, MAE: 0.6200471103002154, MAPE: 0.1533468717723979
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 1, 2), RMSE: 1.2701694022771608, MAE: 0.9099292222280351, MAPE: 0.21810613061548265
2/2 [=

In [2]:
# Full TFT

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, Flatten, MultiHeadAttention
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.callbacks import EarlyStopping
from itertools import product
import gc

# Load dataset
file_path = 'data/basic.csv'
data = pd.read_csv(file_path)

# Remove unnecessary columns
data = data.drop(columns=['last_name, first_name'])

# Handle missing values (e.g., replace with 0)
data = data.fillna(0)

# Extract player_ids that exist in 2019, 2020, 2021, 2022, 2023
data_2019 = data[data['year'] == 2019]
data_2020 = data[data['year'] == 2020]
data_2021 = data[data['year'] == 2021]
data_2022 = data[data['year'] == 2022]
data_2023 = data[data['year'] == 2023]

player_ids_2019 = set(data_2019['player_id'].unique())
player_ids_2020 = set(data_2020['player_id'].unique())
player_ids_2021 = set(data_2021['player_id'].unique())
player_ids_2022 = set(data_2022['player_id'].unique())
player_ids_2023 = set(data_2023['player_id'].unique())

common_player_ids = player_ids_2019 & player_ids_2020 & player_ids_2021 & player_ids_2022 & player_ids_2023

# Extract data for common player_ids
common_data = data[data['player_id'].isin(common_player_ids)]

# Extract data for the years 2019, 2020, 2021, and 2022
final = common_data[common_data['year'].isin([2019, 2020, 2021, 2022])]
final = final.sort_values(by=['player_id', 'year'])

# Select necessary columns (excluding year)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# Separate independent and dependent variables
X = final[features].values
y = final[target].values

# Data scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Convert to time series data
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 4  # Set sequence length
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# Set training data to be the same as the entire data
X_train, y_train = X_seq, y_seq

# Set hyperparameters
state_sizes = [10, 20, 40, 80, 160, 240, 320]
dropout_rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.9]
minibatch_sizes = [32, 64, 128]
learning_rates = [0.0001, 0.001, 0.01]
max_gradient_norms = [0.01, 1, 100.0]
num_heads = [1, 2, 3, 4]

# Generate hyperparameter combinations
hyperparameter_combinations = list(product(state_sizes, dropout_rates, minibatch_sizes, learning_rates, max_gradient_norms, num_heads))

best_rmse = float('inf')
best_mae = float('inf')
best_mape = float('inf')
best_params = None

# Define TFT model (remove static features)
class GatedResidualNetwork(tf.keras.layers.Layer):
    def __init__(self, input_dim, state_size, dropout_rate):
        super(GatedResidualNetwork, self).__init__()
        self.dense1 = Dense(state_size, activation="relu")
        self.dense2 = Dense(input_dim)  # Adjust to input dimension
        self.gate = Dense(input_dim, activation="sigmoid")
        self.layer_norm = LayerNormalization()
        self.dropout = Dropout(dropout_rate)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dropout(x)
        x = self.dense2(x)
        gate_output = self.gate(inputs)
        gated_output = x * gate_output + inputs
        return self.layer_norm(gated_output)

# Modify TFT model to pass input_dim to GatedResidualNetwork
class TFTModel(Model):
    def __init__(self, seq_length, feature_dim, num_heads, ff_dim, state_size, dropout_rate):
        super(TFTModel, self).__init__()

        # Temporal Attention Layers
        self.multi_head_attention1 = MultiHeadAttention(num_heads=num_heads, key_dim=feature_dim)
        self.layer_norm1 = LayerNormalization()

        self.multi_head_attention2 = MultiHeadAttention(num_heads=num_heads, key_dim=feature_dim)
        self.layer_norm2 = LayerNormalization()

        # Gated Residual Networks for gating mechanisms
        self.grn1 = GatedResidualNetwork(feature_dim, state_size, dropout_rate)
        self.grn2 = GatedResidualNetwork(feature_dim, state_size, dropout_rate)

        # Feed Forward Network
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(feature_dim)
        ])

        # Prediction Layers
        self.flatten = Flatten()
        self.dense1 = Dense(state_size, activation="relu")
        self.dropout1 = Dropout(dropout_rate)
        self.dense2 = Dense(state_size // 4, activation="relu")
        self.dense3 = Dense(1, activation="linear")

    def call(self, inputs):
        # Temporal Attention Layer 1 with Residual Connection and Gating
        attn_output1 = self.multi_head_attention1(inputs, inputs)
        out1 = self.layer_norm1(inputs + attn_output1)
        out1 = self.grn1(out1)

        # Temporal Attention Layer 2 with Residual Connection and Gating
        attn_output2 = self.multi_head_attention2(out1, out1)
        out2 = self.layer_norm2(out1 + attn_output2)
        out2 = self.grn2(out2)

        # Feed-Forward Network
        ffn_output = self.ffn(out2)

        # Flatten and predict
        flat_output = self.flatten(ffn_output)

        # Final Dense Layers
        dense_output1 = self.dense1(flat_output)
        drop_output1 = self.dropout1(dense_output1)
        dense_output2 = self.dense2(drop_output1)

        return self.dense3(dense_output2)

# Hyperparameter tuning and training
for state_size, dropout_rate, minibatch_size, learning_rate, max_gradient_norm, num_heads in hyperparameter_combinations:
    tf.keras.backend.clear_session()
    
    # Create model instance
    model = TFTModel(seq_length=seq_length, feature_dim=X_train.shape[2], num_heads=num_heads, ff_dim=32, state_size=state_size, dropout_rate=dropout_rate)
    
    # Compile
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=max_gradient_norm)
    model.compile(
        loss="mse",
        optimizer=optimizer,
        metrics=["accuracy"]
    )

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Train model
    history = model.fit(X_train, y_train, epochs=500, batch_size=minibatch_size, validation_split=0.2, verbose=0, callbacks=[early_stopping])
    
    # Filter 2019 data
    data_19 = common_data[common_data['year'] == 2019]

    # Scale 2019 data
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # Convert to time series data
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # Predict 2019 data
    y_pred_scaled = model.predict(X_2019_seq)

    # Inverse scale
    y_pred = scaler_y.inverse_transform(y_pred_scaled)

    # Restore actual 2019 p_era values for comparison
    y_test_actual = data_19[target].values

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test_actual[:len(y_pred)], y_pred))

    # Calculate MAE
    mae = mean_absolute_error(y_test_actual[:len(y_pred)], y_pred)
    
    # Calculate MAPE
    mape = mean_absolute_percentage_error(y_test_actual[:len(y_pred)], y_pred)

    if rmse < best_rmse:
        best_rmse = rmse
        best_mae = mae
        best_mape = mape
        best_params = (state_size, dropout_rate, minibatch_size, learning_rate, max_gradient_norm, num_heads)

    print(f"Params: {state_size, dropout_rate, minibatch_size, learning_rate, max_gradient_norm, num_heads}, RMSE: {rmse}, MAE: {mae}, MAPE: {mape}")

    del model, history, y_pred_scaled, y_pred  # Explicitly delete necessary variables
    tf.keras.backend.clear_session()
    gc.collect()
    
print(f"Best RMSE: {best_rmse}, Best MAE: {best_mae}, Best MAPE: {best_mape}, Best Params: {best_params}")

2/2 [==============================] - 0s 14ms/step
Params: (10, 0.1, 32, 0.0001, 0.01, 1), RMSE: 0.7535780804288152, MAE: 0.5163386338854593, MAPE: 0.1250820247106295
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 0.01, 2), RMSE: 0.8769627063766863, MAE: 0.6039521494365873, MAPE: 0.1410708076077122
2/2 [==============================] - 0s 2ms/step
Params: (10, 0.1, 32, 0.0001, 0.01, 3), RMSE: 1.2542822021696556, MAE: 0.9227649231562538, MAPE: 0.21491423381724045
2/2 [==============================] - 0s 16ms/step
Params: (10, 0.1, 32, 0.0001, 0.01, 4), RMSE: 0.7209510820270506, MAE: 0.5169610999879384, MAPE: 0.1240232691864838
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 1, 1), RMSE: 1.0553354261384196, MAE: 0.7833876546223958, MAPE: 0.19024234295495704
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 1, 2), RMSE: 1.1185094174446746, MAE: 0.8380099345010423, MAPE: 0.20365449989090412
2/